In [ ]:
from fastapi import APIRouter, HTTPException
from api.config import OLLAMA_URL
from api.schemas.chat import *
from api.services.chat import *
from api.services.llm_loaders import *

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate

# Instantiate router
router = APIRouter(
    prefix="/api/chat", tags=["chat"], responses={404: {"description": "Not found"}},
)

@router.post("/", response_model=ChatResponse, status_code=200)
async def main(request: ChatRequest):
    memory = request.history

    prompt_template = PromptTemplate.from_template("role: {role}\ncontent: {content}")

    chat_prompt_template = FewShotPromptTemplate(
        examples=memory,
        example_prompt=prompt_template,
        suffix="role: {role}\ncontent: {content}",
        input_variables=["role", "content"] 
    )

    chat_prompt = chat_prompt_template.invoke(
        {
            "role": "user",
            "content": request.prompt
        }
    )

    try:
        llm = load_llm_ollama(request.model, base_url=OLLAMA_URL)
        response = llm.invoke(chat_prompt)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
    
    return ChatResponse(history=memory, prompt=request.prompt, response=response)


In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

memory = [
    {
        "role": "ai",
        "content": "Hello! How can I help you today?"
    },
    {
        "role": "human",
        "content": "Hello, My name is John Doe"
    },
    {
        "role": "ai",
        "content": "Hello, John Doe! How can I help you today?"
    }
]

# Convert to list of tuples
memory = [(entry["role"], entry["content"]) for entry in memory]

print(memory)

[('ai', 'Hello! How can I help you today?'), ('human', 'Hello, My name is John Doe'), ('ai', 'Hello, John Doe! How can I help you today?')]


In [ ]:
system_prompt = "You are a chat assistant called LoLLa3, Local LLama3 Assistant."

# Create a flat list with system prompt, memory messages, and the human input
all_messages = [("system", system_prompt)] + memory + [("human", "{input}")]

print(all_messages)

In [33]:
chat_template = ChatPromptTemplate.from_messages(all_messages)

prompt = "Who am I?"

chat_prompt = chat_template.invoke(
    {
        "input": prompt
    }
)

print(chat_prompt)

messages=[SystemMessage(content='You are a chat assistant called LoLLa3, Local LLama3 Assistant.'), AIMessage(content='Hello! How can I help you today?'), HumanMessage(content='Hello, My name is John Doe'), AIMessage(content='Hello, John Doe! How can I help you today?'), HumanMessage(content='Who am I?')]


In [ ]:


llm = ""

chain = chat_prompt | llm
chain.invoke({"input": "What is "})

In [21]:
memory = [
    {
        "role": "ai",
        "content": "Hello! How can I help you today?"
    },
    {
        "role": "human",
        "content": "Hello, My name is John Doe"
    },
    {
        "role": "ai",
        "content": "Hello, John Doe! How can I help you today?"
    }
]

prompt_template = PromptTemplate.from_template("{role}:{content}")

In [19]:
print(prompt_template.invoke(memory[0]).to_string())

ai:Hello! How can I help you today?


In [24]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=memory,
    example_prompt=prompt_template,
    suffix="role: {role}\ncontent: {content}",
    input_variables=["role", "content"]
)

print(
    prompt.invoke(
        {
            "role": "user",
            "content": "What is my name?"
        }
    ).to_string()
)

ai:Hello! How can I help you today?

human:Hello, My name is John Doe

ai:Hello, John Doe! How can I help you today?

role: user
content: What is my name?
